### ***GOOGLE DRIVE AND SYSTEM PATH***

In [1]:
#pip3 install google.colab
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
import sys
path_to_module = 'C:\\Users\\PAPASOFT INC\\Desktop\\SOFT_SIL'
sys.path.append(path_to_module)
import os
#os.environ["OMP_NUM_THREADS"] = '6'

### ***NEEDED LIBRARIES***

In [2]:
# SET DEVICE
import torch
############
# LOAD DATASET
from New_Code_Files.Datasets_Functions import createHashMap, getHashMap, appendInHashMap, deleteFromHashMap, updateHashMap, functionGetDataset, makeHashMapWithListsToHashmapWithDicts
############
# VISUALIZATION
import matplotlib.colors as mcolors
import numpy as np
from New_Code_Files.Visualization import Visualization
############
# CREATE AUTOENCODER
from New_Code_Files.Autoencoder import Autoencoder, createAutoencoder
############

### ***SET DEVICE***

In [3]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### ***LOAD DATASET***

In [4]:
hashMapFilePath = path_to_module+"/New_Code_Files/"
#updateHashMap("R3",{'batch_size': 100, 'n_clusters': 3, 'module_name': 'datasets.datasets_New', 'function_name': 'load_R3_dataloader', 'option_name': ''},hashMapFilePath)
hashMap = getHashMap(hashMapFilePath)
print(hashMap.keys())
#datasetName = "STL10"
#datasetName = "eMNIST_balanced_letters"
datasetName = "R3"
datasetProperties = hashMap[datasetName]
print(datasetProperties)
batchSize = datasetProperties['batch_size'] #= 30
nClusters = datasetProperties['n_clusters'] #= 3
dataloader, inputDim, data_np, labels = functionGetDataset(datasetProperties)
print("Data_Shape is:",inputDim)
print("Batches Number is:",len(dataloader))

dict_keys(['10x73k', 'TCGA', 'coil20', 'eMNIST_balanced_letters', 'eMNIST_mnist', 'eMNIST_balanced_digits', 'Dermatology', 'Ecoil', 'Iris', 'Rings', 'Wine', 'Australian', 'Moons', 'Squeezed_Gauss', 'Gauss_Densities', 'Pendigits', 'fashionMNIST', '3DSpheres', '20_NewsGroups', 'COIL20', 'CIFAR10', 'STL10', 'R15', 'R3'])
{'batch_size': 100, 'n_clusters': 3, 'module_name': 'datasets.datasets_New', 'function_name': 'load_R3_dataloader', 'option_name': ''}
Data_Shape is: 2
Batches Number is: 15


### ***PARAMETERS***

In [5]:
# Latent Dimension
latentDim = 2

negativeSlope = 0

# Batch Size and Number of Clusters
batchSize = datasetProperties['batch_size'] #= 30
nClusters = datasetProperties['n_clusters'] #= 3

# Pre-Training Epochs and Learning Rate
preTrEpochs = 100
preTrLR = 1e-3

# Lamda, Training Epochs and Learning Rate
lamda = 0.01 # Possible values: 0.01, 0.1, 1
# For Balanced_Digits 0.01 seems good
# For Balanced_Letters 0.01 seems good
trLR = 1e-4
trEpochs = 100

kmeans_initialization = True
pretrain = True
isCD = False
num_of_channels = 1
kmeans_n_init = 100

### ***CREATE AUTOENCODER***

In [6]:
autoencoder = createAutoencoder(pathToModule = path_to_module, datasetName =  datasetName,
 inputDim = inputDim, negativeSlope = negativeSlope,
 nClusters = nClusters, device = device, preTrEpochs = preTrEpochs, dataloader = dataloader,
 batchSize = batchSize, preTrLR = preTrLR, lamda = lamda,
 trEpochs = trEpochs, trLR = trLR,
 latentDim = latentDim, isCD = isCD, kmeans_n_init = 100,  num_of_channels = num_of_channels)

 ### ***PRETRAIN AUTOENCODER***

In [7]:
if(pretrain):
    autoencoder.pretrain_autoencoder()
    autoencoder.save_pretrained_weights()
else:
    model_save_path = autoencoder.dataDirPath + "\\Weigths\\autoencoder_weights.pth"
    autoencoder.load_state_dict(torch.load(model_save_path))
    normPreTrEpochs = autoencoder.preTrEpochs  
    autoencoder.preTrEpochs = 1
    autoencoder.pretrain_autoencoder()
    autoencoder.preTrRpochs = normPreTrEpochs

Epoch: 1/100, Loss: 4.337663
Epoch: 2/100, Loss: 1.802372
Epoch: 3/100, Loss: 1.702556
Epoch: 4/100, Loss: 1.647093
Epoch: 5/100, Loss: 1.619609
Epoch: 6/100, Loss: 1.642943
Epoch: 7/100, Loss: 1.568805
Epoch: 8/100, Loss: 1.582652
Epoch: 9/100, Loss: 1.573087
Epoch: 10/100, Loss: 0.960052
Epoch: 11/100, Loss: 0.633669
Epoch: 12/100, Loss: 0.546551
Epoch: 13/100, Loss: 0.481240
Epoch: 14/100, Loss: 0.411779
Epoch: 15/100, Loss: 0.406191
Epoch: 16/100, Loss: 0.406610
Epoch: 17/100, Loss: 0.318611
Epoch: 18/100, Loss: 0.319094
Epoch: 19/100, Loss: 0.409561
Epoch: 20/100, Loss: 0.421267
Epoch: 21/100, Loss: 0.418405
Epoch: 22/100, Loss: 0.340378
Epoch: 23/100, Loss: 0.353415
Epoch: 24/100, Loss: 0.411754
Epoch: 25/100, Loss: 0.361499
Epoch: 26/100, Loss: 0.336041
Epoch: 27/100, Loss: 0.336449
Epoch: 28/100, Loss: 0.338599
Epoch: 29/100, Loss: 0.304547
Epoch: 30/100, Loss: 0.262428
Epoch: 31/100, Loss: 0.237654
Epoch: 32/100, Loss: 0.261523
Epoch: 33/100, Loss: 0.312614
Epoch: 34/100, Loss

### ***VISUALIZATION***

In [8]:
color_list = list(mcolors.CSS4_COLORS.keys()) + list(mcolors.XKCD_COLORS.keys())
np.random.shuffle(color_list)
color_list = ['deepskyblue', 'gold', 'hotpink', 'limegreen'] + color_list
visualization = Visualization(color_list,15,autoencoder)

 ### ***TRAINING WITH SOFT SILHOUETTE***

In [9]:
if(kmeans_initialization):
    autoencoder.kmeans_initialization(100)

C:\ProgramData\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


In [10]:
"""
while True:
    # Code block to execute repeatedly
    autoencoder.trEpochs = 10
    autoencoder.train_autoencoder(silhouette_method = "default")
    visualization.makeExcel()
    #visualization.makePlot(useTSNE=False)
    #visualization.makePlot(useTSNE=True)
    visualization.plot_3D()

    # Check the condition to exit the loop
    if autoencoder.experimentName == "Experiment_400":
        break
"""
autoencoder.train_autoencoder(silhouette_method = "default")

visualization.makeExcel()
#visualization.makePlot(useTSNE=False)
visualization.plot_tsne()

TypeError: Evaluation.autoencoder_evaluation() missing 1 required positional argument: 'autoencoder_labels'